# Diffusion

In [2]:
#%pylab inline
import random
import pickle as pkl
import numpy as np

## Chargement des données

In [3]:
def load_file (path='') :
    # Lecture du fichier contenant les épisodes
    monFichier = open (path, 'r')
    data = monFichier.readlines()
    data = np.array(data)
    monFichier.close()

    return data

In [4]:
def build_graphe (data) :
    #Initialisation du graphe
    graphe = {} #graphe
    graphe['s'] = {} #successeurs
    graphe['p'] = {} #predecesseurs
    users = {}
    preds = {}
    relations = []
    
    # Chaque ligne est un épisode
    for l in data :
        episode = l.replace('\r\n', '')
        episode = l.split(';')
        cascade = []
        
        #On recupère les infections d'un episode. Chaque couple user:time est une infection
        for infect in episode :
         
            splited_infect = infect.split(':')
            
            #non_vide
            if (len(splited_infect) == 2) :
                #On sépare l'utilisateur et le temps
                user = int(splited_infect[0])
                time = float(splited_infect[1])
                users[user] = 1
                #print user
                # l'utilisateur n'existe pas encore dans la liste de predecesseurs
                if (not graphe['p'].has_key(user)) :
                    graphe['p'][user] = []
                
                if (len(cascade) > 0) :
                    i = np.array(cascade)[:,0]
                    # l'utilisateur n'existe pas encore dans la liste de successeurs
                    for infected_user in i :
                        #print infected_user
                        if (not graphe['s'].has_key(int(infected_user))) :
                            graphe['s'][int(infected_user)] = []
                        graphe['p'][user].append(infected_user)
                        graphe['s'][int(infected_user)].append(int(user))
                        graphe['p'][user] = list(set(graphe['p'][user]))
                        graphe['s'][int(infected_user)] =list(set(graphe['s'][int(infected_user)]))
                        
                cascade.append(np.array([user, time]))
        cascade = np.array(cascade).reshape(len(cascade), 2)
        
        for u in cascade[:,0] :
            if (not u in preds) :
                preds[u] = {}
            i = np.where(cascade[:,0] == u)[0]
            pred = np.where(cascade[:,1] < cascade[i, 1])[0]
            if pred.shape[0] > 0:
                pred = cascade[pred,0]
                preds[u][len(relations)] = pred
            else:
                preds[u][len(relations)] = []

        relations.append(cascade)
        
    return relations, graphe, users, preds


In [5]:
def getD (data, graphe, path = '', dataset = 'train') :
        
        D = {}
        # D+ = { cascade | {u, v} € cascade, Tv = Tu+1}
        # D- = { cascade | u € cascade OR {u, v} € cascade, Tv > Tu+1}
        
        for noeud in graphe['s'].keys() :
            
            # S'il n'existe pas encore dans D
            if (not D.has_key(noeud)):
                D[noeud] = {}
                
            for successeur in graphe['s'][noeud] :
                # Si D ne contient pas les ensembles pour u et v
                if (not D[noeud].has_key(successeur)):
                    D[noeud][successeur] = getDuv(noeud, successeur, data)
                    
        monFichier = open(path + dataset, 'w')
        pkl.dump (D, monFichier)
        monFichier.close()
        
        return D

In [6]:
def getDuv (noeud, successeur, data) :
    
        Dplus = []
        Dmoins = []
        
        for i, d in enumerate(data) :
            u = np.where(d[:,0] == noeud)
            v = np.where(d[:,0] == successeur)
            if u[0].size > 0 and v[0].size > 0:
                if d[u[0][0],1] < d[v[0][0],1]:
                    Dplus.append(i)
            if u[0].size > 0:
                if (v[0].size > 0 and d[u[0][0], 1] >= d[v[0][0], 1]) or v[0].size == 0:
                    Dmoins.append(i)
        r = {}
        r['plus'] = Dplus
        r['minus'] = Dmoins
        return r

In [ ]:
data = load_file('cascades_train.txt')
train, graphe_train, users_train, preds_train = np.array (build_graphe (data))
#print graphe_train
#print (users_train)

In [28]:
data = load_file('cascades_test.txt')
test, graphe_test, users_test, preds_test = np.array (build_graphe (data))

5000
5000


## Modèle Independent Cascades

In [ ]:
infection_init = 0.1
likelihood = 1e-4
likelihoods = []

"""
# Apprentissage
while (len(likelihoods) < 2) :
    i = 0
    for noeud in graphe_train['s'].keys():
        for s in graphe['s'][noeud]:
            p[noeud][s] = d_likelihood_process(noeud, s)
            # Calcul de la vraissemblance
            res = 0
            D_bis = D[noeud][s]['p']
            for d in D_bis :
                noeud = int(noeud)
                key = str(int(noeud))+"__"
                
                
                


            r = numerator / (len(self.D[node][successor]['plus']) + len (self.D[node][successor]['minus']))
        i = i + 1
    t = t + 1

    likelihoods.append(likelihood_process(data))
    print str(self.t) + ' => likelihood : ' + str (self.likelihood[-1])
    
"""